<a href="https://colab.research.google.com/github/laws-spatial/NTCNA/blob/main/notebooks/Census_Data_Calls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas census

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 30.5 MB/s eta 0:00:00


In [ ]:
# import libraries
import pandas as pd #work with tabular data
import os #used to set working directory in Google Drive
import requests # web service accesses
import geopandas as gpd # spatial data
from datetime import datetime #make fields date
from census import Census
import pickle
from google.colab import files

%matplotlib inline

#Set directory as Google Drive
wrkDir = "/content/drive/MyDrive/Native_American_Tribal_Needs_Assessment_Code"
os.chdir(wrkDir)

In [ ]:
%run "/content/drive/MyDrive/Native_American_Tribal_Needs_Assessment_Code/script/helpers.py"

In [ ]:
with open('./data/api_key.txt') as key:
    api_key=key.read().strip()

print(api_key)

83dd3d2dbaca15f49b987d0aa4e24856d0cc8ac9


In [ ]:
# village = [santee, rosalie, macy, walthill, winnebego]
vil_codes = [43475, 42250, 30170, 51245, 53275]

#years 2011 - 2021
years = range(2011, 2022)

#set global constants
stFIPS = "31"

c = Census(api_key)

In [ ]:
# census code tables
code_table = ["A", "B", "C", "D", "E", "F", "G", "I"]

# tables of interest
tables = ["B17001", "B01001", "B01002"]

# dictionarys of column suffixes
B01002_col_suf = ["_001E"]
B17001_col_suf = ["_001E", "_002E", "_004E", "_005E", "_006E", "_007E", "_008E",
               "_009E", "_018E", "_019E", "_020E", "_021E","_022E", "_023E"]

race_col_suf = {"B01001": ['_001E', '_003E', '_004E', '_005E', '_006E', '_007E',
                           '_008E', '_009E', '_010E', '_011E', '_012E', '_013E', 
                           '_014E', '_015E', '_016E', '_018E', '_019E', '_020E',
                           '_021E', '_022E', '_023E', '_024E', '_025E', '_026E', 
                           '_027E', '_028E', '_029E', '_030E', '_031E'],
                "B01002": B01002_col_suf,
                "B17001": B17001_col_suf}
totpop_col_suf = {"B01001": ['_001E', '_003E', '_004E', '_005E', '_006E', '_007E',
                              '_008E', '_009E', '_010E', '_011E', '_012E', '_013E', 
                              '_014E', '_015E', '_016E', '_017E', '_018E', '_019E', 
                              '_020E', '_021E', '_022E', '_023E', '_024E', '_025E', 
                              '_027E', '_028E', '_029E', '_030E', '_031E', '_032E', 
                              '_033E', '_034E', '_035E', '_036E', '_037E', '_038E', 
                              '_039E', '_040E', '_041E', '_042E', '_043E', '_044E', 
                              '_045E', '_046E', '_048E', '_047E', '_049E'],
                "B01002": B01002_col_suf,
                "B17001": B17001_col_suf}

In [ ]:
# get columns
col = Construct_columns(code_table, tables=tables, race_col_suf=race_col_suf, totpop_col_suf=totpop_col_suf)

cols = col.construct_columns()

In [ ]:
# blank dataframes
places = pd.DataFrame([])
state = pd.DataFrame()
us = pd.DataFrame()

# request
for year in years:
  try:
    # send request to census bureau for state and US info
    acs5_resp_state = pd.DataFrame(c.acs5.state(cols, stFIPS, year=year))
    acs5_resp_us = pd.DataFrame(c.acs5.us(cols, year=year))

    # add year
    acs5_resp_state["year"] = year
    acs5_resp_us["year"] = year

    # concat dataframes
    state = pd.concat([state, acs5_resp_state], axis=0)
    us = pd.concat([us, acs5_resp_us], axis=0)

  except Exception as e:
      print(f"US-State {str(year)} had an exception: {e}")

  for vil in vil_codes:
    try:
      #create call
      place = str(vil)

      #send request to census bureau
      acs5_resp = c.acs5.state_place(cols, stFIPS, place, year=year)
 
      # convert response to dataframe
      vill = pd.DataFrame.from_dict(acs5_resp)
      
      vill["year"] = year
      vill["entityID"] = int(vil)

      #concat dataframe
      places = pd.concat([places, vill], axis = 0)
      
    except Exception as e:
      print(f"Place {str(year)} had an exception: {e}")
  print(f"{str(year)} done")

2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
US-State 2021 had an exception: Geography is not available in 2021. Available years include (2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)
Place 2021 had an exception: Geography is not available in 2021. Available years include (2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)
Place 2021 had an exception: Geography is not available in 2021. Available years include (2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)
Place 2021 had an exception: Geography is not available in 2021. Available years include (2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)
Place 2021 had an exception: Geography is not available in 2021. Available years include (2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)
Place 2021 had an exception: Geography is not available in 2021. Available years includ

Remove negative numbers

In [ ]:
dataframes = [places, state, us]
for df in dataframes:
  for col in list(df.columns.values):
    if df[col].dtype == "float64":
      df[col].where(df[col] > 0, 0, inplace=True)
  else:
    pass

Copy dataframes

In [ ]:
places2 = places.copy()
state2 = state.copy()
us2 = us.copy()

In [ ]:
labels_dict = {
    "total": "",
    "white":"A",
    "black": "B",
    "nat": "C",
    "asian": "D",
    "pacisl": "E",
    "other": "F",
    "mult": "G",
    "his": "I"
}

In [ ]:
class Census_calc:
  def __init__(self, df, df_label, labels_dict=labels_dict):
    self.df = df 
    self.df_label = df_label
    self.labels_dict = labels_dict 

  def agegroup_calc(self):
    df = self.df
    df_label = self.df_label
    labels_dict = self.labels_dict

    for race, label in labels_dict.items():
      if race == "total":
        # under 18
        df[f"{df_label}_age_total_und18"] = round(df[["B01001_003E", "B01001_004E", "B01001_005E", "B01001_006E", 
                                                  "B01001_027E", "B01001_028E", "B01001_029E", "B01001_030E"]]
                                          .sum(axis=1) / df["B01001_001E"] * 100, 1)
        # 18-64
        df[f"{df_label}_age_total_18_64"] = round(df[["B01001_007E", "B01001_008E", "B01001_009E", "B01001_010E", 
                                                  "B01001_011E", "B01001_012E", "B01001_013E", "B01001_014E", 
                                                  "B01001_015E", "B01001_016E", "B01001_017E", "B01001_018E",
                                                  "B01001_019E", "B01001_031E", "B01001_032E", "B01001_033E", 
                                                  "B01001_034E", "B01001_035E", "B01001_036E", "B01001_037E", 
                                                  "B01001_038E", "B01001_039E", "B01001_040E", "B01001_041E", 
                                                  "B01001_042E", "B01001_043E"]]
                                          .sum(axis=1) / df["B01001_001E"] * 100, 1)
        # 65+
        df[f"{df_label}_age_total_ov65"] = round(df[["B01001_020E", "B01001_021E", "B01001_022E", "B01001_023E", 
                                                  "B01001_024E", "B01001_025E", "B01001_044E", "B01001_045E", 
                                                  "B01001_046E", "B01001_047E", "B01001_048E", "B01001_049E"]]
                                          .sum(axis=1) / df["B01001_001E"] * 100, 1)
        # total population
        df[f"{df_label}_pop_total"] = df[f"B01001_001E"].astype(int)
      else:
        # under 18
        df[f"{df_label}_age_{race}_und18"] = round(df[[f"B01001{label}_003E", f"B01001{label}_004E", f"B01001{label}_005E", f"B01001{label}_006E", 
                                                    f"B01001{label}_018E", f"B01001{label}_019E", f"B01001{label}_020E", f"B01001{label}_021E"]]
                                            .sum(axis=1) / df[f"B01001{label}_001E"] * 100, 1)
        # 18-64
        df[f"{df_label}_age_{race}_18_64"] = round(df[[f"B01001{label}_007E", f"B01001{label}_008E", f"B01001{label}_009E", f"B01001{label}_010E",
                                                    f"B01001{label}_011E", f"B01001{label}_012E", f"B01001{label}_013E", f"B01001{label}_021E", 
                                                    f"B01001{label}_022E", f"B01001{label}_023E", f"B01001{label}_024E", f"B01001{label}_025E", 
                                                    f"B01001{label}_026E", f"B01001{label}_027E", f"B01001{label}_028E"]]
                                            .sum(axis=1) / df[f"B01001{label}_001E"] * 100, 1)
        # 65+
        df[f"{df_label}_age_{race}_ov65"] = round(df[[f"B01001{label}_014E", f"B01001{label}_015E", f"B01001{label}_016E", f"B01001{label}_029E",
                                                    f"B01001{label}_030E", f"B01001{label}_031E"]]
                                            .sum(axis=1) / df[f"B01001{label}_001E"] * 100, 1)
        # total population
        df[f"{df_label}_pop_{race}"] = df[f"B01001{label}_001E"].astype(int)

    return df

  def medage_calc(self):
    df = self.df
    df_label = self.df_label
    labels_dict = self.labels_dict

    for race, label in labels_dict.items():
      df[f"{df_label}_medage_{race}"] = round(df[f"B01002{label}_001E"], 1)

    return df
  
  def poverty_calc(self):
    df = self.df
    df_label = self.df_label
    labels_dict = self.labels_dict
    
    for race, label in labels_dict.items():
      # percentage of total
      df[f"{df_label}_pov_{race}_tot"] = round(df[f"B17001{label}_002E"] / df[f"B17001{label}_001E"] * 100, 1)
      # under 18 percentage
      df[f"{df_label}_pov_{race}_18"] = round(df[[f"B17001{label}_004E", f"B17001{label}_005E", f"B17001{label}_006E", f"B17001{label}_007E", 
                                              f"B17001{label}_008E", f"B17001{label}_009E", f"B17001{label}_018E", f"B17001{label}_019E", 
                                              f"B17001{label}_020E", f"B17001{label}_021E", f"B17001{label}_022E", f"B17001{label}_023E"]]
                                      .sum(axis=1) / df[f"B17001{label}_001E"] * 100, 1)
      
    return df

  def perform_calcs(self):
    df = self.df
    df = self.agegroup_calc()
    df = self.medage_calc()
    df = self.poverty_calc()

    return df

In [ ]:
def subset_cols(df, col_pre):
  if col_pre == "pl":
    cols = ["NAME", "year", "entityID"]
  else:
    cols = ["NAME", "year"]
  col_pre = col_pre + "_"
  more_cols = [col for col in df.columns.values if col[0:3] == f"{col_pre}"]
  cols.extend(more_cols)

  df = df[cols]

  return df

In [ ]:
# places
new_calc_pl = Census_calc(places2, "pl")
places2 = new_calc_pl.perform_calcs()
places2 = subset_cols(places2, "pl")

# state
new_calc_st = Census_calc(state2, "st")
state2 = new_calc_st.perform_calcs()
state2 = subset_cols(state2, "st")

# us
new_calc_us = Census_calc(us2, "us")
us2 = new_calc_us.perform_calcs()
us2 = subset_cols(us2, "us")

In [ ]:
places2.shape

In [ ]:
state2.shape

In [ ]:
us2.shape

**Merge US and state to places**

In [ ]:
# merge us to places
print(places2.shape)
places_us = places2.merge(us2, how="inner", on="year")
# print(places_us.shape)

(50, 66)


In [ ]:
places_us_st = places_us.merge(state2, how="inner", on="year")
places_us_st = places_us_st.fillna(0)
# print(places_us_st.shape)

**Bring in geometries and merge tabular data**

In [ ]:
#create merge key
places_us_st["PLACEFIPS"] = "31" + places_us_st["entityID"].astype(str)
places_us_st = places_us_st.copy()
places_us_st = places_us_st.drop(["NAME_x", "NAME_y", "NAME"], axis=1)
# print(places_us_st.columns.values)

In [ ]:
#bring in places geometries
places_geos = gpd.read_file("https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_Populated_Places/FeatureServer/0/query?where=PLACEFIPS%20%3D%20'3143475'%20OR%20PLACEFIPS%20%3D%20'3142250'%20OR%20PLACEFIPS%20%3D%20'3130170'%20OR%20PLACEFIPS%20%3D%20'3138750'%20OR%20PLACEFIPS%20%3D%20'3151245'%20OR%20PLACEFIPS%20%3D%20'3148900'%20OR%20PLACEFIPS%20%3D%20'3153275'&outFields=NAME,STFIPS,PLACEFIPS&outSR=4326&f=json")
places_geos = places_geos[["NAME", "PLACEFIPS", "geometry"]]


In [ ]:
# merge places to geos and keep all rows
print(places_geos.shape)
places_full = places_geos.merge(places_us_st, how="right", on="PLACEFIPS")
print(places_full.shape)

(7, 3)
(50, 194)


In [ ]:
# convert year from integer to datetime
places_full['year'] = pd.to_datetime(places_full["year"].astype(str) + "-06-01")
# places_full["year"]

In [ ]:
places_full["year"]

0    2011-06-01
1    2011-06-01
2    2011-06-01
3    2011-06-01
4    2011-06-01
5    2012-06-01
6    2012-06-01
7    2012-06-01
8    2012-06-01
9    2012-06-01
10   2013-06-01
11   2013-06-01
12   2013-06-01
13   2013-06-01
14   2013-06-01
15   2014-06-01
16   2014-06-01
17   2014-06-01
18   2014-06-01
19   2014-06-01
20   2015-06-01
21   2015-06-01
22   2015-06-01
23   2015-06-01
24   2015-06-01
25   2016-06-01
26   2016-06-01
27   2016-06-01
28   2016-06-01
29   2016-06-01
30   2017-06-01
31   2017-06-01
32   2017-06-01
33   2017-06-01
34   2017-06-01
35   2018-06-01
36   2018-06-01
37   2018-06-01
38   2018-06-01
39   2018-06-01
40   2019-06-01
41   2019-06-01
42   2019-06-01
43   2019-06-01
44   2019-06-01
45   2020-06-01
46   2020-06-01
47   2020-06-01
48   2020-06-01
49   2020-06-01
Name: year, dtype: datetime64[ns]

In [ ]:
# black_cols = places_full.filter(like='age_black').columns.values.tolist()
# black_cols
# places_full[black_cols].head(10)

In [ ]:
# places_full[["pl_age_black_und18",	"pl_age_black_18_64",	"pl_age_black_ov65"]]

Change data type of year

In [ ]:
df = gpd.read_file("data/ne_ntcna_places_census_updated_all.geojson")

In [ ]:
from datetime import datetime
df["year"] = pd.to_datetime(df["year"])
df["year_int"] = df["year"].dt.strftime("%Y-%m-%d")
df.drop("year", axis=1, inplace=True)

In [ ]:
df.head()

,NAME,PLACEFIPS,entityID,pl_age_tot_und18,pl_age_tot_18_64,pl_age_tot_ov65,pl_pop_tot,pl_age_white_und18,pl_age_white_18_64,pl_age_white_ov65,...,st_pov_pacisl_tot,st_pov_pacisl_18,st_pov_other_tot,st_pov_other_18,st_pov_mult_tot,st_pov_mult_18,st_pov_his_tot,st_pov_his_18,geometry,year_int
0,Santee,3143475,43475,44.9,54.5,0.6,334.0,0.0,94.4,5.6,...,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-97.85201 42.84192, -97.85185 42.841...",2011-06-01
1,Rosalie,3142250,42250,24.7,53.7,21.6,231.0,13.8,59.6,26.6,...,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.50927 42.06107, -96.50932 42.060...",2011-06-01
2,Macy,3130170,30170,53.5,42.4,4.1,851.0,0.0,15.4,84.6,...,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.37321 42.12605, -96.37278 42.125...",2011-06-01
3,Pender,3138750,38750,21.6,55.8,22.6,1152.0,21.5,57.1,23.4,...,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.71631 42.11889, -96.71632 42.117...",2011-06-01
4,Walthill,3151245,51245,33.7,54.9,11.4,861.0,10.1,61.2,28.7,...,50.8,32.0,23.3,11.5,23.7,14.3,24.3,12.7,"POLYGON ((-96.49161 42.15520, -96.49145 42.151...",2011-06-01


0   2011-06-01
1   2011-06-01
2   2011-06-01
3   2011-06-01
4   2011-06-01
Name: year, dtype: datetime64[ns]

**Export to json**


In [ ]:
# Export geojson
places_full.to_file("data/ne_ntcna_places_census_updated_all.geojson")
files.download("data/ne_ntcna_places_census_updated_all.geojson")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
class Construct_columns:
  def __init__(self, code_table, tables, race_col_suf, totpop_col_suf):
    self.code_table=code_table 
    self.tables=tables 
    self.race_col_suf=race_col_suf 
    self.totpop_col_suf=totpop_col_suf

  def _totpop_columns(self):
    col_suf_list = self.totpop_col_suf
    tables = self.tables
    cols = []
    for table in tables:
      col_suffixes = col_suf_list[table]
      new_cols = [f"{table}{col_suf}" for col_suf in col_suffixes]
      cols.extend(new_cols)

    return cols
  
  def _single_race_columns(self, table_code, race_code):
    col_suffixes = self.race_col_suf[table_code]
    cols = [f"{table_code}{race_code}{suf}" for suf in col_suffixes]
    return cols

  def construct_columns(self):
    cols = ["NAME"]
    cols.extend(self._totpop_columns())
    for table in self.tables:
      for code in self.code_table:
        new_cols = self._single_race_columns(table, code)
        cols.extend(new_cols)

    return cols